In [ ]:
import itertools

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install keras

In [ ]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 111.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.9 MB/s eta 0:00:00


In [ ]:
import random
from keras.utils import to_categorical
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, Bidirectional,TimeDistributed
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
import xml.etree.ElementTree as ET
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mutual_info_score
import csv
import re
import numpy as np
from keras.models import load_model
import unicodedata
import os
# fix random seed for reproducibility
import random
import numpy as np
import torch

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import operator
from sklearn.metrics import f1_score

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.decomposition import TruncatedSVD
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from numpy.random import seed
from keras.models import load_model
seed(1)

In [ ]:

pivot_num=200
pivot_min_st=10
word_vector_size=256
topWords=10000
max_review_len=500
hidden_units_num=256
dataSize=1000
testSize=200
src_count = 20
un_count = 40
src='books'
dest='music'

dataSize=1000
testSize=200
pivot_min_st=10
train=[]
train_target=[]
test=[]

target=[]
c=0
i=0
pivotsCounts= []
names = []
tran=[]
tokens=[]
transyn=[]

In [ ]:
def GetTopNMI(n,CountVectorizer,X,target):
    MI = []
    length = X.shape[1]


    for i in range(length):
        temp=mutual_info_score(X[:, i], target)
        MI.append(temp)
    MIs = sorted(range(len(MI)), key=lambda i: MI[i])[-n:]
    return MIs,MI


In [ ]:
def getCounts(X,i):

    return (sum(X[:,i]))

In [ ]:
def mean_pred(y_true, y_pred):
    return f1_score(y_true, y_pred, average='weighted')

#seperating the pivots to unigrams and bigrams features
def get_unigram_bigram(my_list):
    unigram = []
    bigram = []
    for feature in my_list:
        if (len(feature.split()) == 1):
            unigram.append(feature)
        if (len(feature.split()) == 2):
            word=feature.split()
            temp=word[0]+'_'+word[1]
            bigram.append(temp)
    return unigram,bigram

#initializng pivot to index dictionary
def fill_pivot_dict(names):
    i=1
    pivot2int = dict()
    for name in names:
        pivot2int[name]=i
        i=i+1
    return  pivot2int

#fix some encoding issues
def getClear_full(sentence):
    sentence = " ".join(sentence)
    r = re.findall(r'\b\w+\b', sentence.lower())
    r = " ".join(r)
    # r = (r.decode('unicode_escape').encode('ascii', 'ignore'))
    return r

def getClearList(sentences):
    clearSentences = []
    for s in sentences:
        clearSentences.append(getClear_full(s))
    return  clearSentences


In [ ]:
def concatenate_neg_and_pos(neg,pos):
    data_1 = []
    data_2 = []

# Read the first CSV file
    with open(neg, 'r', newline='', encoding='utf-8') as csvfile1:
        reader = csv.reader(csvfile1)
        for row in reader:
            data_1.append(row)

# Read the second CSV file
    with open(pos, 'r', newline='', encoding='utf-8') as csvfile2:
        reader = csv.reader(csvfile2)
        for row in reader:
            data_2.append(row)

# Combine the data from both files
    reviews = data_1 + data_2
    return reviews

In [ ]:
pivotsCounts= []
unlabeled = []
names = []

filename = "/content/drive/My Drive/code/AE-PA-SCL/"+src + "_to_" + dest + "/split/"
if not os.path.exists(os.path.dirname(filename)):
    reviews=concatenate_neg_and_pos("/content/drive/My Drive/code/AE-PA-SCL/English/"+src+"/negative.csv","/content/drive/My Drive/code/AE-PA-SCL/English/"+src+"/positive.csv")
    n=pd.read_csv("/content/drive/My Drive/code/AE-PA-SCL/English/"+src+"/negative.csv",header=None,encoding='utf-8')
    p=pd.read_csv("/content/drive/My Drive/code/AE-PA-SCL/English/"+src+"/positive.csv",header=None,encoding='utf-8')
    test_data_neg = random.sample(range(0, dataSize), testSize)
    test_data_pos = random.sample(range(dataSize, 2*dataSize), testSize)
    random_array = np.concatenate((test_data_neg, test_data_pos))
    train = []
    test = []
    test_target = []
    train_target = []
    for i in range(0, 2*dataSize):
        if i in random_array:
            test.append(reviews[i])
            target = 0 if i < dataSize else 1
            test_target.append(target)
        else:
            train.append(reviews[i])
            target = 0 if i < dataSize else 1
            train_target.append(target)
else:
    with open("/content/drive/My Drive/code/AE-PA-SCL/"+src + "_to_" + dest + "/split/train", 'rb') as f:
        train = pickle.load(f)
    with open("/content/drive/My Drive/code/AE-PA-SCL/"+src + "_to_" + dest + "/split/test", 'rb') as f:
        test = pickle.load(f)
    with open("/content/drive/My Drive/code/AE-PA-SCL/"+src + "_to_" + dest + "/split/train_target", 'rb') as f:
        train_target = pickle.load(f)
    with open("/content/drive/My Drive/code/AE-PA-SCL/"+src + "_to_" + dest + "/split/test_target", 'rb') as f:
        test_target = pickle.load(f)


In [ ]:
# sets x train matrix for classification
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=5,binary=True)
flattened_train = [' '.join(sublist) for sublist in train]
X_2_train = bigram_vectorizer.fit_transform(flattened_train).toarray()

In [ ]:
# gets all the train and test for pivot classification
unlabelled=concatenate_neg_and_pos("/content/drive/My Drive/code/AE-PA-SCL/English/"+src+"/"+src+"UN.csv",'/content/drive/My Drive/code/AE-PA-SCL/musickan.csv')
source=pd.read_csv("/content/drive/My Drive/code/AE-PA-SCL/English/"+src+"/"+src+"UN.csv",header=None,encoding='utf-8') #english labeled data
target=pd.read_csv('/content/drive/My Drive/code/AE-PA-SCL/musickan.csv',header=None,encoding='utf-8') # target unlabeld data

train_df = pd.DataFrame(train)
source=np.append(source, train_df, axis=0)
#source = pd.concat([source, train_df], ignore_index=True)

#source=source+train
src_count = 20
un_count = 40

In [ ]:
unlabelled = np.append(source, target, axis=0)

In [ ]:
bigram_vectorizer_source = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=src_count, binary=True)
flattened_source = [' '.join(sublist) for sublist in source]
X_2_train_source = bigram_vectorizer_source .fit_transform(flattened_source).toarray()

In [ ]:
pip install scikit-learn indic-nlp-library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 134.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 7.4 MB/s eta 0:00:00


In [ ]:
#from langdetect import detect
from sklearn.feature_extraction import DictVectorizer

# Sample tokenized Kannada data
#kannada_tokens = [
   # ["ಈ", "ಒಂದು", "ಕನ್ನಡ", "ಪಠ್ಯವನ್ನು"],
   # ["ಇದು", "ಕನ್ನಡ", "ಪಠ್ಯವನ್ನು", "ಅರ್ಥಮಾಡಿಕೊಳ್ಳಲು", "ಪರಿಕ್ಷೆ"],
   ## ["ಕನ್ನಡ", "ಪಠ್ಯವು", "ನಮಗೆ", "ಸಹಾಯ", "ಮಾಡುತ್ತದೆ"]
#]
target=pd.read_csv('/content/drive/My Drive/code/AE-PA-SCL/musickan.csv',header=None,encoding='utf-8',names=['text'])

import nltk
#natural language tool kit
# Tokenize the texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(target)
sequences = tokenizer.texts_to_sequences(target)

# Get word index
word_index = tokenizer.word_index
print(word_index)
wn = nltk.WordNetLemmatizer()
#lemmatize is a function inside lemmatizer
#example print(wn.lemmatize('cactus'))
#print(wn.lemmatize('cacti'))
# gives = cactus
def lemmatization(token_txt):
    text = [wn.lemmatize(word) for word in token_txt ]
    return text

import re
import string

punctuation='''!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\n'''
def remove_punctuation(txt):
    txt_nopunct ="".join([c for c in txt if c not in punctuation])
    txt_nopunct = re.sub('\w*\d\w*', '', txt_nopunct)
    return txt_nopunct

target['text1'] = target['text'].apply(lambda x: remove_punctuation(x))


from indicnlp.tokenize import indic_tokenize

def tokenize(txt):
    tokens =re.split(' ',txt)
    return tokens

target['text2'] = target['text1'].apply(lambda x: tokenize(x.lower()))


#tokens=kan_data['kan_tweets_tokens']
def clean(txt):
    for i in txt:
        if len(i)==1:
            del(i)
    return txt
target['text3'] = target['text2'].apply(lambda x: clean(x))
#print(target['text3'])
print(target['text3'].head())
# Convert tokenized data to a list of dictionaries
token_counts = []
for tokens in target['text3']:
    #print(tokens)
    token_count_dict = {}
    for token in tokens:
        #print(token_count_dict)
        if token in token_count_dict:
            token_count_dict[token] += 1
        else:
            token_count_dict[token] = 1
    token_counts.append(token_count_dict)


# Create an instance of DictVectorizer
vectorizer = DictVectorizer(sparse=False)

# Fit and transform the token counts
X_target = vectorizer.fit_transform(token_counts)

# Convert to an array for inspection
#X_array = X.toarray()

# Get feature names (vocabulary)
feature_names = vectorizer.get_feature_names_out()
for i in range(0,len(feature_names)):
    print(feature_names[i])
# Print the feature names and the array
  #print("Feature Names:", feature_names)
print("Feature Array:\n", X_target)


<>:37: SyntaxWarning: invalid escape sequence '\w'
<>:37: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipython-input-3165939416.py:37: SyntaxWarning: invalid escape sequence '\w'
  txt_nopunct = re.sub('\w*\d\w*', '', txt_nopunct)


Streaming output truncated to the last 5000 lines.
ಮುಸುಕು
ಮೂಕ
ಮೂಕರನ್ನಾಗಿ
ಮೂಕರನ್ನಾಗಿಸಿದೆ
ಮೂಕವಿಸ್ಮಿತರಾಗುವುದಿಲ್ಲ
ಮೂಗಿನ
ಮೂಗೇಟುಗಳನ್ನು
ಮೂಡತೊಡಗಿತು
ಮೂಡಿ
ಮೂಡಿತು
ಮೂಡಿಬಂದಿವೆ
ಮೂಡಿಯಾಗಿ
ಮೂಡಿಸಲು
ಮೂಡಿಸುತ್ತದೆ
ಮೂಡ್
ಮೂಡ್‌ಗಳನ್ನು
ಮೂಡ್‌ಗೆ
ಮೂಡ್‌ನಲ್ಲಿದ್ದರೆ
ಮೂಡ್‌ನಲ್ಲಿದ್ದೇನೆ
ಮೂನ್‌ನಲ್ಲಿ
ಮೂರನೆಯ
ಮೂರನೆಯದನ್ನು
ಮೂರನೆಯದು
ಮೂರನೇ
ಮೂರರಲ್ಲಿ
ಮೂರು
ಮೂರುಟ್ರ್ಯಾಕ್
ಮೂರುಮಾರ್ಗದ
ಮೂರುವಿಭಾಗದ
ಮೂರೂವರೆ
ಮೂರ್ಖ
ಮೂರ್ಖತನ
ಮೂರ್ಖತನದ
ಮೂರ್ಖತನದಿಂದ
ಮೂರ್ಖರಂತೆ
ಮೂರ್ಖರನ್ನಾಗಿಸುತ್ತದೆ
ಮೂರ್ಖರಾಗಿರಿ
ಮೂರ್ಖರೆಂದು
ಮೂರ್ಛೆ
ಮೂರ್ಛೆಗೊಳಿಸಲು
ಮೂಲ
ಮೂಲಕ
ಮೂಲಕವೂ
ಮೂಲಕ್ಕಿಂತ
ಮೂಲಕ್ಕೆ
ಮೂಲಗಳಿಂದ
ಮೂಲತಃ
ಮೂಲದಂತೆ
ಮೂಲದಲ್ಲಿ
ಮೂಲಭೂತ
ಮೂಲಭೂತವಾಗಿ
ಮೂಲಭೂತವಾಗಿದೆ
ಮೂಲವನ್ನು
ಮೂಲವಲ್ಲದ
ಮೂಲವಾಗಿದೆ
ಮೂಲವು
ಮೂಲವೂ
ಮೂಲೆ
ಮೂಲೆಯ
ಮೂಲೆಯಲ್ಲಿ
ಮೂಳೆ
ಮೂಳೆಗಳನ್ನು
ಮೂಳೆಗಳು
ಮೂಳೆಯವರೆಗೆ
ಮೂವತ್ತು
ಮೂವರ
ಮೂವರೊಂದಿಗೆ
ಮೂವಿಗಳನ್ನು
ಮೂವ್
ಮೃಗೀಯವಾಗಿದೆ
ಮೃದು
ಮೃದುಗೊಳಿಸಲಾಗುತ್ತದೆ
ಮೃದುತ್ವ
ಮೃದುತ್ವವು
ಮೃದುವಾಗಿ
ಮೃದುವಾದ
ಮೆ
ಮೆಕ್ಕಾರ್ಟ್ನಿಎಸ್ಕ್ಯೂ
ಮೆಕ್ಯಾನಿಕಲ್
ಮೆಗಾಲೊಮೇನಿಯಾಕ್
ಮೆಚ್ಚಬೇಕು
ಮೆಚ್ಚಿದ
ಮೆಚ್ಚಿದರೂ
ಮೆಚ್ಚಿದೆ
ಮೆಚ್ಚಿನ
ಮೆಚ್ಚಿನವನ್ನು
ಮೆಚ್ಚಿನವಲ್ಲದಿದ್ದರೂ
ಮೆಚ್ಚಿನವು
ಮೆಚ್ಚಿನವುಗಳ
ಮೆಚ್ಚಿನವುಗಳಲ್ಲ
ಮೆಚ್ಚಿನವುಗಳಲ್ಲದಿದ್ದರೂ
ಮೆಚ್ಚಿನವುಗಳಲ್ಲಿ
ಮೆಚ್ಚಿನವುಗಳವರೆಗೆ
ಮೆಚ್ಚಿನವುಗಳಾಗಿ
ಮ

In [ ]:
#from langdetect import detect
from sklearn.feature_extraction import DictVectorizer

# Sample tokenized Kannada data
#kannada_tokens = [
   # ["ಈ", "ಒಂದು", "ಕನ್ನಡ", "ಪಠ್ಯವನ್ನು"],
   # ["ಇದು", "ಕನ್ನಡ", "ಪಠ್ಯವನ್ನು", "ಅರ್ಥಮಾಡಿಕೊಳ್ಳಲು", "ಪರಿಕ್ಷೆ"],
   ## ["ಕನ್ನಡ", "ಪಠ್ಯವು", "ನಮಗೆ", "ಸಹಾಯ", "ಮಾಡುತ್ತದೆ"]
#]
#target=pd.read_csv('/content/drive/My Drive/my code/musickan.csv',header=None,encoding='utf-8',names=['text'])

import nltk
#natural language tool kit

wn = nltk.WordNetLemmatizer()
#lemmatize is a function inside lemmatizer
#example print(wn.lemmatize('cactus'))
#print(wn.lemmatize('cacti'))
# gives = cactus
def lemmatization(token_txt):
    text = [wn.lemmatize(word) for word in token_txt ]
    return text

import re
import string

punctuation='''!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\n'''
def remove_punctuation(txt):
    txt_nopunct ="".join([c for c in txt if c not in punctuation])
    txt_nopunct = re.sub('\w*\d\w*', '', txt_nopunct)
    return txt_nopunct
unlabelled = pd.Series(unlabelled[:, 0])
unlabel = unlabelled.apply(lambda x: remove_punctuation(x))


from indicnlp.tokenize import indic_tokenize

def tokenize(txt):
    tokens =re.split(' ',txt)
    return tokens

unlabel1 = unlabel.apply(lambda x: tokenize(x.lower()))


#tokens=kan_data['kan_tweets_tokens']
def clean(txt):
    for i in txt:
        if len(i)==1:
            del(i)
    return txt
unlabel2 = unlabel1.apply(lambda x: clean(x))
#print(target['text3'])
#print(unlabel2.head())
# Convert tokenized data to a list of dictionaries
token_count = []
for tokens in unlabel2:
    #print(tokens)
    token_count_dict1 = {}
    for token in tokens:
        #print(token_count_dict)
        if token in token_count_dict1:
            token_count_dict1[token] += 1
        else:
            token_count_dict1[token] = 1
    token_count.append(token_count_dict1)


# Create an instance of DictVectorizer
vectorizer_unlabel = DictVectorizer(sparse=False)

# Fit and transform the token counts
X_unlabel = vectorizer_unlabel.fit_transform(token_count)

# Convert to an array for inspection
#X_array = X.toarray()

# Get feature names (vocabulary)
feature_names_unlabel = vectorizer_unlabel.get_feature_names_out()
for i in range(0,len(feature_names_unlabel)):
    print(feature_names_unlabel[i])
# Print the feature names and the array
  #print("Feature Names:", feature_names)
print("Feature Array:\n", X_unlabel)


<>:30: SyntaxWarning: invalid escape sequence '\w'
<>:30: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipython-input-1195310089.py:30: SyntaxWarning: invalid escape sequence '\w'
  txt_nopunct = re.sub('\w*\d\w*', '', txt_nopunct)



	
		
			houses
			lo
			the
		calcio
		calling
		expressions
		il
		italian
		la
		made
		meals
		professional
		the
		time
		waiting
	a
	after
	against
	all
	america
	american
	an
	and
	april
	are
	as
	at
	attention
	attracting
	authentic
	bad
	basic
	businesses
	celebrations
	china
	compliments
	conservatives
	creativity
	dating
	demonstrated
	did
	does
	driving
	earlier
	eye
	family
	fighting
	for
	form
	hal
	have
	how
	i
	il
	im
	in
	innovation
	is
	italy
	its
	la
	large
	le
	lies
	ma
	management
	mermaids
	metrics
	millhone
	monetary
	more
	navigating
	nestorian
	networks
	number
	on
	one
	pat
	peoples
	physical
	pig
	pro
	ravitch
	sadly
	seek
	should
	simple
	so
	speaking
	strong
	the
	then
	there
	this
	though
	tinkerbell
	today
	tu
	v
	was
	ways
	we
	what
	whats
	when
	where
	while
	who
	why
	will
a
aa
aaaaa
aafto
aahed
aaii
aaiicoms
aaiis
aand
aarp
aarrgggh
ab
aba
abaci
aback
abackthis
abagnale
abagnales
abandon
abandoned
abandoning
abandonment
abandonmentkozols
abandonmentsh

In [ ]:
#get a sorted list of pivots with respect to the MI with the label
MIsorted,RMI=GetTopNMI(2000,CountVectorizer,X_2_train,train_target)
MIsorted.reverse()
c=0
i=0

In [ ]:
import sys

c=0
i=0
pivotsCounts=[]
names=[]
while (c<pivot_num):

    name= bigram_vectorizer.get_feature_names_out()[MIsorted[i]]

    s_count = getCounts(X_2_train_source,list(bigram_vectorizer_source.get_feature_names_out()).index(name)) if name in bigram_vectorizer_source.get_feature_names_out() else 0

    if(s_count>=pivot_min_st):
        names.append(name)
        if name in vectorizer_unlabel.get_feature_names_out():
            pivotsCounts.append(list(vectorizer_unlabel.get_feature_names_out()).index(name))
            c+=1
            print("feature is ",name," it MI is ",RMI[MIsorted[i]]," in source ",s_count,"",c)
    i+=1

feature is  boring  it MI is  0.021876559025580014  in source  205  1
feature is  waste  it MI is  0.01939164609914009  in source  153  2
feature is  excellent  it MI is  0.01892523564868507  in source  482  3
feature is  t  it MI is  0.01828357682895032  in source  2766  4
feature is  disappointing  it MI is  0.017306293741166618  in source  74  5
feature is  no  it MI is  0.01499058752857374  in source  1445  6
feature is  not  it MI is  0.014737718195052207  in source  3516  7
feature is  don  it MI is  0.014345637830058405  in source  1108  8
feature is  bad  it MI is  0.013840381166620813  in source  461  9
feature is  great  it MI is  0.012024886075085506  in source  1743  10
feature is  was  it MI is  0.011334158998978716  in source  3321  11
feature is  wonderful  it MI is  0.010816891904836451  in source  406  12
feature is  instead  it MI is  0.010334888029314954  in source  379  13
feature is  unfortunately  it MI is  0.009621908690745883  in source  169  14
feature is  easy

In [ ]:
pip install nltk

In [ ]:
pip install googletrans

In [ ]:
pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.7 MB/s eta 0:00:00


In [ ]:
import sys
from googletrans import Translator
from contextlib import redirect_stdout
from deep_translator import GoogleTranslator
translator = Translator()
trans=[]
for x in names:
    try:
        # Attempt to translate the phrase
        trans.append(GoogleTranslator(source='en',target='kn').translate(x))
    except TranslationNotFound:
        # If translation not found, print a message and skip the phrase
        print(f"Translation not found for:{x}. Skipping...")
        trans.append(x)# Append the original phrase or a placeholder

print(names,trans)


['boring', 'waste', 'excellent', 't', 'disappointing', 'no', 'not', 'don t', 'don', 'a must', 'waste of', 'bad', 'your money', 't waste', 'an excellent', 'great', 'was', 'waste your', 'wonderful', 'instead', 'a waste', 'unfortunately', 'highly recommend', 'easy', 'poorly', 'weak', 'disappointed', 'a wonderful', 'must read', 'i', 'i highly', 'didn t', 'didn', 'nothing', 'loved it', 'much better', 'recommend', 'highly', 'worst', 'dull', 'the best', 'a great', 'money', 'boring and', 'beautifully', 'poor', 'is no', 'too', 'highly recommended', 'better', 'stupid', 'two stars', 'enjoyed this', 'if', 'great book', 'trying', 'doesn t', 'doesn', 'finish', 'way too', 'at all', 'trying to', 'reading for', 'poorly written', 'care about', 'and easy', 'loved', 'i was', 'did', 'flat', 'recommend this', 'because', 'love', 'best', 't put', 'gives', 'writing', 'gotten', 'but', 'ridiculous', 'love this', 'is a', 'have been', 'i don', 'sorry', 'supposed', 'whatsoever', 'anything', 'much', 'would', 'worse'

In [ ]:
from collections import Counter

def count_word_frequencies(trans, arrays_of_tokens):
    # Flatten the list of lists into a single list of tokens
    all_tokens = [token for sublist in arrays_of_tokens for token in sublist]

    # Filter the tokens to keep only those present in the trans set
    filtered_tokens = [token for token in trans if token in all_tokens]

    # Use Counter to count the frequencies
    word_frequencies = Counter(filtered_tokens)

    return word_frequencies,filtered_tokens

# Example usage
arrays_of_tokens = target['text3']

print(trans)

frequencies,tokens = count_word_frequencies(trans, arrays_of_tokens)
print(frequencies)


['ನೀರಸವಾದ', 'ವ್ಯರ್ಥ', 'ಅತ್ಯುತ್ತಮ', 'ಟಿ', 'ನಿರಾಶೆಯ', 'ಇಲ್ಲ', 'ಇಲ್ಲ', 'ಡಾನ್ ಟಿ', 'ಡಾನ್', 'ಎ ಕಡ್ಡಾಯ', 'ವ್ಯರ್ಥ', 'ಕೆಟ್ಟ', 'ನಿಮ್ಮ ಹಣ', 'ಟಿ ತ್ಯಾಜ್ಯ', 'ಅತ್ಯುತ್ತಮ', 'ಶ್ರೇಷ್ಠ', 'ಸದ್ದಿಲ್ಲದೆ', 'ನಿಮ್ಮ ವ್ಯರ್ಥ', 'ಅದ್ಭುತ', 'ಬದಲಾಗಿ', 'ಒಂದು ತ್ಯಾಜ್ಯ', 'ದುರದೃಷ್ಟವಶಾತ್', 'ಹೆಚ್ಚು ಶಿಫಾರಸು ಮಾಡಿ', 'ಸುಲಭವಾದ', 'ಕಳಪೆಯಾಗಿರುವ', 'ದುರ್ಬಲ', 'ನಿರಾಶೆಗೊಂಡ', 'ಅದ್ಭುತ', 'ಓದಬೇಕು', 'ನಾನು', 'ನಾನು ಹೆಚ್ಚು', 'ಮಾಡಲಿಲ್ಲ', 'ಮಾಡಲಿಲ್ಲ', 'ಯಾವುದೂ ಇಲ್ಲ', 'ಇಷ್ಟವಾಯಿತು', 'ಹೆಚ್ಚು ಉತ್ತಮವಾಗಿದೆ', 'ಶಿಫಾರಸು ಮಾಡು', 'ಬಹಳ', 'ಕೆಟ್ಟ', 'ಮಂದಬುದ್ಧಿಯ', 'ಅತ್ಯುತ್ತಮ', 'ಒಂದು ದೊಡ್ಡದು', 'ಹಣ', 'ನೀರಸ ಮತ್ತು', 'ಸುಂದರವಾಗಿ', 'ಬಡ', 'ಇಲ್ಲ', 'ಕೂಡಾ', 'ಹೆಚ್ಚು ಶಿಫಾರಸು ಮಾಡಲಾಗಿದೆ', 'ಉತ್ತಮ', 'ಮೂರ್ಖ', 'ಎರಡು ನಕ್ಷತ್ರಗಳು', 'ಇದನ್ನು ಆನಂದಿಸಿದೆ', 'ಇತ್ತು', 'ದೊಡ್ಡ ಪುಸ್ತಕ', 'ಪ್ರಯತ್ನಿಸುವುದು', 'ಟಿ ಆಗುವುದಿಲ್ಲ', 'ಡು', 'ಮುಗಿಸು', 'ದಾರಿ ತುಂಬಾ', 'ಎಲ್ಲ', 'ಮಾಡಲು ಪ್ರಯತ್ನಿಸುತ್ತಿದೆ', 'ಓದುವಿಕೆ', 'ಕಳಪೆಯಾಗಿ ಬರೆಯಲಾಗಿದೆ', 'ಬಗ್ಗೆ ಕಾಳಜಿ ವಹಿಸಿ', 'ಮತ್ತು ಸುಲಭ', 'ಪ್ರೀತಿಸಿದ', 'ನಾನು', 'ಮಾಡಿದ', 'ಚಪ್ಪಟೆ', 'ಇದನ್ನು ಶಿಫಾರಸು ಮಾಡಿ', 'ಕಾರಣ', 'ಪ್ರೀತಿಸು', 'ಅತ್ಯುತ್ತಮ', 'ಟಿ ಪುಟ್', 'ಕೊಡುವುದು', 'ಬರೆಯುವುದು', 'ಮುಟ್ಟಿದ', 'ಆದರೆ',

In [ ]:
print(len(pivotsCounts))

200


In [ ]:
c=0
i=0
tran=[]


for token in tokens:
    t_count = getCounts(X_target,list(vectorizer.get_feature_names_out()).index(token)) if token in vectorizer.get_feature_names_out() else 0
    # Iterate over each token in the 'tokens'
    if(t_count>=pivot_min_st):

        if token not in tran:

            if token in vectorizer.get_feature_names_out():

                tran.append(token)
                pivotsCounts.append(list(vectorizer_unlabel.get_feature_names_out()).index(token))
                c += 1
                print("feature is",token,"in target",t_count,"",c)
i += 1

print(pivotsCounts)

feature is ಅತ್ಯುತ್ತಮ in target 245.0  1
feature is ಇಲ್ಲ in target 86.0  2
feature is ಕೆಟ್ಟ in target 68.0  3
feature is ಶ್ರೇಷ್ಠ in target 64.0  4
feature is ಅದ್ಭುತ in target 86.0  5
feature is ದುರದೃಷ್ಟವಶಾತ್ in target 15.0  6
feature is ದುರ್ಬಲ in target 13.0  7
feature is ನಾನು in target 2280.0  8
feature is ಮಾಡಲಿಲ್ಲ in target 12.0  9
feature is ಬಹಳ in target 63.0  10
feature is ಸುಂದರವಾಗಿ in target 16.0  11
feature is ಉತ್ತಮ in target 260.0  12
feature is ಮಾಡಿದ in target 63.0  13
feature is ಕಾರಣ in target 22.0  14
feature is ಆದರೆ in target 930.0  15
feature is ಎ in target 27.0  16
feature is ಇವೆ in target 10.0  17
feature is ಯಾವುದಾದರೂ in target 13.0  18
feature is ಬಹು in target 10.0  19
feature is ಕೇವಲ in target 110.0  20
feature is ಅಲ್ಲಿ in target 45.0  21
feature is ಅಥವಾ in target 284.0  22
feature is ಅಷ್ಟೇ in target 10.0  23
feature is ಹೆಚ್ಚು in target 451.0  24
feature is ಉತ್ತಮವಾಗಿದೆ in target 38.0  25
feature is ಸರಳವಾದ in target 18.0  26
feature is ಚೆನ್ನಾಗಿ in target 71.0  27
feature

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
import sys
from googletrans import Translator
from contextlib import redirect_stdout
from deep_translator import GoogleTranslator
translator = Translator()
trans=[]
pivot_words=[]
word_pairs=[]

def find_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    if set(synonyms):
        return set(synonyms)
    else: return 'none'

for word in names:
    kantrans=(GoogleTranslator(source='en',target='kn').translate(word))  #kannada translation of 'word'
    #print(word)                                                          #synonym of 'word'
    synonyms = find_synonyms(word)
    c=0
    for item in synonyms:
        if item in vectorizer_unlabel.get_feature_names_out():
           if item not in names:
            #print(item)
                transyn.append(item)
                pivotsCounts.append(list(vectorizer_unlabel.get_feature_names_out()).index(item))
    if synonyms=='none':
        word_pair=(kantrans,word)
    else:
        word_pair=(kantrans,word,synonyms)

    word_pairs.append(word_pair)

for pair in word_pairs:
    print(pair)

print(len(word_pairs))



[nltk_data] Downloading package wordnet to /root/nltk_data...


('ನೀರಸವಾದ', 'boring', {'oil_production', 'ho-hum', 'drill', 'irksome', 'tedious', 'dull', 'tiresome', 'wearisome', 'bore', 'drilling', 'boring', 'tire', 'deadening', 'slow'})
('ವ್ಯರ್ಥ', 'waste', {'pine_away', 'lay_waste_to', 'thriftlessness', 'wasteland', 'desolate', 'waste_product', 'macerate', 'wastefulness', 'blow', 'dissipation', 'squander', 'knock_off', 'rot', 'do_in', 'neutralise', 'liquidate', 'scourge', 'barren', 'ravage', 'waste', 'godforsaken', 'run_off', 'languish', 'neutralize', 'consume', 'waste_material', 'devastate', 'wild', 'ware', 'emaciate', 'waste_matter', 'permissive_waste'})
('ಅತ್ಯುತ್ತಮ', 'excellent', {'first-class', 'splendid', 'excellent', 'fantabulous'})
('ಟಿ', 't', {'thyroxin', 'deoxythymidine_monophosphate', 'liothyronine', 'triiodothyronine', 'tonne', 'T', 'tetraiodothyronine', 'thymine', 'MT', 't', 'thyroxine', 'metric_ton'})
('ನಿರಾಶೆಯ', 'disappointing', {'disappointing', 'let_down', 'dissatisfactory', 'unsatisfying', 'disappoint'})
('ಇಲ್ಲ', 'no', {'no_more'

In [ ]:
print(len(pivotsCounts))


2654


In [ ]:
print(len(X_unlabel))

8089


In [ ]:
#takes out fifth of the training data for validation(with respect to the represantation learning task)
source_valid = len(source)//5
target_valod = len(target)//5
c=0
y = X_unlabel[:,pivotsCounts]

In [ ]:
x =np.delete( X_unlabel, pivotsCounts, 1)  # delete second column of C

In [ ]:
x.shape

(8089, 70578)

In [ ]:


x_valid=np.concatenate((x[:source_valid][:], x[-target_valod:][:]), axis=0)

x = x[source_valid:-target_valod][:]


    #we take fifth of the source examples and fifth of the target examples to keep the same ratio in validation
y_valid = np.concatenate((y[:source_valid][:], y[-target_valod:][:]), axis=0)

y = y[source_valid:-target_valod][:]


In [ ]:
filename ="/content/drive/My Drive/code/AE-PA-SCL/"+ src+"_to_"+dest+"/"+"pivot_names/pivot_names_"+src+"_"+dest+"_"+str(pivot_num)+"_"+str(pivot_min_st)
if not os.path.exists(os.path.dirname(filename)):
        os.makedirs(os.path.dirname(filename))


with open(filename, 'wb') as f:
    pickle.dump(names, f)
filename = "/content/drive/My Drive/code/AE-PA-SCL/"+ src + "_to_" + dest + "/split/"
if not os.path.exists(os.path.dirname(filename)):
    os.makedirs(os.path.dirname(filename))
    with open("/content/drive/My Drive/code/AE-PA-SCL/"+src + "_to_" + dest + "/split/train", 'wb') as f:
        pickle.dump(train, f)
    with open("/content/drive/My Drive/code/AE-PA-SCL/"+src + "_to_" + dest + "/split/test", 'wb') as f:
        pickle.dump(test, f)
    with open("/content/drive/My Drive/code/AE-PA-SCL/"+src + "_to_" + dest + "/split/train_target", 'wb') as f:
        pickle.dump(train_target, f)
    with open("/content/drive/My Drive/code/AE-PA-SCL/"+src + "_to_" + dest + "/split/test_target", 'wb') as f:
        pickle.dump(test_target, f)
filename ="/content/drive/My Drive/code/AE-PA-SCL/"+ src+"_to_"+dest+"/"+"pivotsCounts/"+"pivotsCounts"+src+"_"+dest+"_"+str(pivot_num)+"_"+str(pivot_min_st)
if not os.path.exists(os.path.dirname(filename)):
        os.makedirs(os.path.dirname(filename))
with open("/content/drive/My Drive/code/AE-PA-SCL/"+src+"_to_"+dest+"/"+"pivotsCounts/"+"pivotsCounts"+src+"_"+dest+"_"+str(pivot_num)+"_"+str(pivot_min_st), 'wb') as f:
    pickle.dump(pivotsCounts, f)


    #finally, we return the training and validation data, there is not test data since we do not care about the test in the representation learning task


In [ ]:
x.shape


(6472, 70578)

In [ ]:
dim=300
inputs=x.shape[1]
outputs = len(pivotsCounts)
HUs =300

In [ ]:
model = Sequential()
model.add(Dense(HUs,kernel_initializer='glorot_normal', input_shape=(inputs,)))
    #model.add(Dense(HUs, input_shape=(inputs,)))
model.add(Activation('sigmoid'))
model.add(Dense(outputs))
model.add(Activation('sigmoid'))
print(model.summary())

lr_schedule = ExponentialDecay(
initial_learning_rate=0.1,
decay_steps=100000,
decay_rate=0.96,
staircase=True)
from tensorflow.keras.losses import BinaryCrossentropy
sgd = SGD(learning_rate=lr_schedule, momentum=0.9)
loss_fn = BinaryCrossentropy(from_logits=False)  # Sigmoid already applied

model.compile(optimizer=sgd, loss=loss_fn, metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 300)            │    21,173,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2654)           │       798,854 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 2654)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,972,554 (83.82 MB)

 Trainable params: 21,972,554 (83.82 MB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
#stops as soon as the validaion loss stops decreasing
earlyStopping = EarlyStopping(monitor='val_loss', patience=0, mode='min')
#saveing only the best model
bestmodel="/content/drive/My Drive/code/AE-PA-SCL/"+src+"_to_"+dest+"/best_model.keras"
save_best = ModelCheckpoint(bestmodel, monitor='val_loss', verbose=0, save_best_only=True, mode='auto')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

    # Generic checkpoint (overwrites every training run)
    generic_checkpt = "/content/drive/My Drive/code/AE-PA-SCL/"+src+"_to_"+dest+"/model_checkpoint.h5.keras"
    generic_checkpoint = ModelCheckpoint(generic_checkpt, save_best_only=True)



    h = model.fit(
        x, y,
        batch_size=20,
        callbacks=[earlyStopping, save_best, generic_checkpoint, seed_checkpoint],
        epochs=10,
        validation_data=(x_valid, y_valid),
        shuffle=True
    )


    print(f"Last validation loss: {h.history['val_loss'][-1]}")

In [ ]:
print(x.shape)
print(y.shape)
print(x_valid.shape)
print(y_valid.shape)

(6472, 70578)
(6472, 2654)
(1617, 70578)
(1617, 2654)


In [ ]:
import os
from tensorflow.keras.models import load_model
import numpy as np

src='books'
dest='music'
pivot_num=200
pivot_min_st=10
HUs=300
weight_str = "/content/drive/My Drive/code/AE-PA-SCL/"+src + "_to_" + dest + "/weights/w_"+src+"_"+dest+"_"+str(pivot_num)+"_"+str(pivot_min_st)+"_"+str(HUs)+".weights.h5"
filename = weight_str
if not os.path.exists(os.path.dirname(filename)):
        os.makedirs(os.path.dirname(filename))
    #saving the entire model
model = load_model(bestmodel)
weights = model.get_weights()
for i, w in enumerate(weights):
    np.save(f"{weight_str}_weight_{i}.npy", w)

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 5 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
weights = model.get_weights()
for i, w in enumerate(weights):
    print(f"Weight {i}: shape {w.shape}")

Weight 0: shape (70578, 300)
Weight 1: shape (300,)
Weight 2: shape (300, 2654)
Weight 3: shape (2654,)


In [ ]:
model.save_weights(weight_str)